In [82]:
!pip3 install sklearn
!pip3 install numpy
!pip3 install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import string

In [84]:
fake_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FakeTrue/Fake5.csv')
true_news = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/FakeTrue/True5.csv')

In [85]:
fake_news.head()

,title,text,source,subject,target,label
0,O ministro da saúde de Israel contraiu o coron...,"Segundo o Times of Israel, Litzman teria queb...",E-Farsa,Político,Falso,FAKE
1,Carlos Bolsonaro publicou em seu perfil do Tw...,Acontece que a foto não foi publicada por Carl...,E-Farsa,Político,Falso,FAKE
2,A Igreja Universal está vendendo álcool ungido...,No mesmo dia em que o assunto se tornou famoso...,E-Farsa,Religioso,Falso,FAKE
3,Os EUA prenderam o doutor que vendeu o coronav...,O doutor Charles Lieber foi realmente preso em...,E-Farsa,Político,Falso,FAKE
4,termo “Black Friday” nasceu na época da escrav...,As origens desse boato são indeterminadas. Enc...,E-Farsa,Entretenimento,Falso,FAKE


In [86]:
true_news.head()

,title,text,source,subject,target,label
0,É verdade que Lula se manifestou contra a inva...,É verdade que Lula se manifestou contra a inva...,Aos Fatos,Político,Verdadeiro,True
1,"PSOL se posicionou contra a lei 2.016/2015, c...","Desde o início, o PSOL se posicionou contra a ...",Aos Fatos,Político,Verdadeiro,True
2,Tebet e Lula pedem que TSE remova campanha vei...,A campanha de Simone Tebet (MDB) e candidato p...,CNN,Político,Verdadeiro,True
3,Português descobre caixa torácica de dinossaur...,"No início de agosto, uma equipe de pesquisador...",CNN,Entretenimento,Verdadeiro,True
4,Onda de calor extremo na Europa será padrão at...,A onda de calor recorde que varreu a Europa es...,CNN,Entretenimento,Verdadeiro,True


In [87]:
true_news.shape,fake_news.shape

((669, 6), (700, 6))

In [88]:
fake_news['class']=0
true_news['class']=1

**Separação de dados para testes manuais**

In [89]:
manual_testing_fake = fake_news.tail(10)
manual_testing_true = true_news.tail(10)

In [90]:
for i in range(700,669,1):
    fake_news.drop([i],axis=0,inplace=True)

for i in range(669,700,1):
    fake_news.drop([i],axis=0,inplace=True)

In [91]:
manual_testing=pd.concat([manual_testing_true,manual_testing_fake],axis=0)
manual_testing.to_csv("/content/drive/MyDrive/Colab Notebooks/FakeTrue/test05.csv")

Pré-processamento de dados

In [92]:
merged_data = pd.concat([true_news,fake_news],axis=0)
merged_data.head()

,title,text,source,subject,target,label,class,target
0,É verdade que Lula se manifestou contra a inva...,É verdade que Lula se manifestou contra a inva...,Aos Fatos,Político,Verdadeiro,True,1,NaN
1,"PSOL se posicionou contra a lei 2.016/2015, c...","Desde o início, o PSOL se posicionou contra a ...",Aos Fatos,Político,Verdadeiro,True,1,NaN
2,Tebet e Lula pedem que TSE remova campanha vei...,A campanha de Simone Tebet (MDB) e candidato p...,CNN,Político,Verdadeiro,True,1,NaN
3,Português descobre caixa torácica de dinossaur...,"No início de agosto, uma equipe de pesquisador...",CNN,Entretenimento,Verdadeiro,True,1,NaN
4,Onda de calor extremo na Europa será padrão at...,A onda de calor recorde que varreu a Europa es...,CNN,Entretenimento,Verdadeiro,True,1,NaN


remover colunas não necessárias para um processamento eficiente 

In [93]:
required_df = merged_data.drop(["title","subject","source"],axis=1)
required_df.head()

,text,target,label,class,target
0,É verdade que Lula se manifestou contra a inva...,Verdadeiro,True,1,NaN
1,"Desde o início, o PSOL se posicionou contra a ...",Verdadeiro,True,1,NaN
2,A campanha de Simone Tebet (MDB) e candidato p...,Verdadeiro,True,1,NaN
3,"No início de agosto, uma equipe de pesquisador...",Verdadeiro,True,1,NaN
4,A onda de calor recorde que varreu a Europa es...,Verdadeiro,True,1,NaN


embaralhar nossos dados

In [94]:
required_df = required_df.sample(frac=1)
required_df.head()

,text,target,label,class,target
641,Trump EXPLODE em colapso matinal acusando o pr...,NaN,FAKE,0,Falso
445,O Ministro da Saúde da África do Sul diz que o...,NaN,FAKE,0,Falso
519,"Lula diz que Auxílio Brasil acaba em dezembro,...",NaN,FAKE,0,Falso
315,senadora berrava gopi gopi durante impeachment...,NaN,FAKE,0,Falso
211,Enquanto a nova primeira-ministra Liz Truss se...,Verdadeiro,True,1,NaN


 limpar nossos dados. Todas as pontuações e caracteres desnecessários, incluindo links de sites, não significam nada para os dados, por isso são removidos antes de serem processados.

In [95]:
def remove_un(data):
    data = data.lower()
    data = re.sub('\[.*?\]','',data)
    data = re.sub('\\W',' ',data)
    data = re.sub('https?://\S+|www.\S+','',data)
    data = re.sub('<.*?>+','',data)
    data = re.sub('[%s]'%re.escape(string.punctuation),'',data)
    data = re.sub('\n','',data)
    data = re.sub('\w*\d\w','',data)
    return data
required_df['text']= required_df['text'].apply(remove_un)
required_df.head()

,text,target,label,class,target
641,trump explode em colapso matinal acusando o pr...,NaN,FAKE,0,Falso
445,o ministro da saúde da áfrica do sul diz que o...,NaN,FAKE,0,Falso
519,lula diz que auxílio brasil acaba em dezembro ...,NaN,FAKE,0,Falso
315,senadora berrava gopi gopi durante impeachment...,NaN,FAKE,0,Falso
211,enquanto a nova primeira ministra liz truss se...,Verdadeiro,True,1,NaN


In [96]:
required_df.to_csv("/content/drive/MyDrive/Colab Notebooks/FakeTrue/preprocessed05.csv")
required_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/FakeTrue/preprocessed05.csv")

**Vectorization**

Vetorização
Vetorização é o processo de mapear nossa linguagem natural em valores numéricos que é usado para fazer previsões usando diferentes algoritmos de aprendizagem de máquina.

Vamos fazer duas variáveis que representam independentes e dependem de variáveis em cálculos numéricos. A equação linear representada por y=mx+c é usada durante o cálculo interno.

In [21]:
x = required_df['text']
y = required_df['class']

Agora vamos dividir os dados de treinamento e teste

In [22]:
x_train, x_test, y_train,y_test=train_test_split(x,y,test_size=0.25)

Usaremos tfidfVectorizer para este projeto. Outros vetorizadores semelhantes como seq_to_seq também podem ser usados

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorization = TfidfVectorizer()
xv_train  = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

Modelo de Treinamento
Regressão Logística
Para este projeto em particular, usaremos regressão logística para treinamento e testes.

Vamos começar importando biblioteca

In [24]:
from sklearn.linear_model import LogisticRegression

**Logistic Regression**

Modelo de treinamento

In [25]:
LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [26]:
LR.score(xv_test,y_test)

0.8151515151515152

O modelo treinado com Regressão Logística é 98% preciso.

Agora vamos olhar mais fundo no desempenho com dados de teste e comparar com o nosso modelo original

In [27]:
pred_LR = LR.predict(xv_test)
print(classification_report(y_test,pred_LR))

              precision    recall  f1-score   support

           0       0.86      0.75      0.80       165
           1       0.78      0.88      0.83       165

    accuracy                           0.82       330
   macro avg       0.82      0.82      0.81       330
weighted avg       0.82      0.82      0.81       330



**Decision Tree Classification**

In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
DT  = DecisionTreeClassifier()
DT.fit(xv_train,y_train)

DecisionTreeClassifier()

In [37]:
DT.score(xv_test,y_test)

0.7484848484848485

In [38]:
pred_DT = DT.predict(xv_test)

In [39]:
print(classification_report(y_test,pred_DT))

              precision    recall  f1-score   support

           0       0.77      0.71      0.74       165
           1       0.73      0.79      0.76       165

    accuracy                           0.75       330
   macro avg       0.75      0.75      0.75       330
weighted avg       0.75      0.75      0.75       330



**Gradient Boosting Classifier**

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [41]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train,y_train)

GradientBoostingClassifier(random_state=0)

In [42]:
GBC.score(xv_test,y_test)

0.8121212121212121

In [43]:
pred_GBC = GBC.predict(xv_test)

In [44]:
print(classification_report(y_test,pred_GBC))

              precision    recall  f1-score   support

           0       0.81      0.81      0.81       165
           1       0.81      0.81      0.81       165

    accuracy                           0.81       330
   macro avg       0.81      0.81      0.81       330
weighted avg       0.81      0.81      0.81       330



**Random Forest Classifier**

In [45]:
from sklearn.ensemble import RandomForestClassifier

In [46]:
RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train,y_train)

RandomForestClassifier(random_state=0)

In [47]:
RFC.score(xv_test,y_test)

0.8212121212121212

In [48]:
pred_RFC = RFC.predict(xv_test)

In [49]:
print(classification_report(y_test,pred_RFC))

              precision    recall  f1-score   support

           0       0.78      0.89      0.83       165
           1       0.87      0.75      0.81       165

    accuracy                           0.82       330
   macro avg       0.83      0.82      0.82       330
weighted avg       0.83      0.82      0.82       330



**Manual Testing**

O mesmo tipo de treinamento e teste pode ser feito usando outros modelos diferentes. Você pode seguir o link para o link original do GitHub para outros algoritmos de classificação. Este GitHub inclui seguir algoritmos

Classificação da Árvore de Decisão
Classificador de aumento de gradiente
Classificador florestal aleatório

Teste manual
Agora vamos olhar para testes manuais que são mais intuitivos e interativos.

In [50]:
def output_label(n):
    if n ==0:
        return "Fake News"
    elif n==1:
        return "True News"
    else:
        return "ERROR!!"
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test['text'] = new_def_test['text'].apply(remove_un)
    new_x_test = new_def_test['text']
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)
    
    return print(f'''\n
                    LR Prediction: {output_label(pred_LR)}\n
                    DT Prediction: {output_label(pred_DT)}\n
                    GBC Prediction: {output_label(pred_GBC)}\n
                    RFC Prediction: {output_label(pred_RFC)}\n
                 ''')

Obtendo texto de entrada do usuário e predindo classificação

In [51]:
news = input("Digite sua mensagem")
manual_testing(news)

Digite sua mensagemrainha da inglaterra morreu


                    LR Prediction: Fake News

                    DT Prediction: True News

                    GBC Prediction: Fake News

                    RFC Prediction: Fake News

                 


Fonte para referência do algoritmo teste: 
https://medium.com/caret-systems/fake-news-detection-with-python-aaf965c24861